In [5]:
from hello import hello_logger
import os

In [17]:
keys = [
    'condenser.output',
    'condenser.pv',
]

In [7]:
fp = 'C:\\Users\\natha\\Documents\\test'
os.chdir(fp)

In [8]:
fn = "15L PID Test 2.csv"
h = hello_logger.HelloLogger('192.168.1.7', 10, fn, keys)
h.start()

In [18]:
h.stop()

### WIP analysis 15L with PID on

In [21]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
xl = Excel()
xl.DisplayAlerts = False
wb = xl.Workbooks.Open(fn)
wb.SaveAs(fn.replace(".csv", ".xlsb"), FileFormat=xlc.xlExcel12)
ws = wb.Worksheets(1)
cells = ws.Cells
cr = cells.Range

In [23]:
def fixdata():
    top = cr("A1")
    bot = top.End(xlc.xlDown)
    bot = top.End(xlc.xlToRight).Offset(bot.Row, 1)

    data = cr(top, bot).Value
    from itertools import zip_longest
    data = list(map(list, zip_longest(*data)))
    rows = len(data[0])
    cols = len(data)
    for i in range(cols):
        cdat = data[i]
        for j in range(rows):
            v = cdat[j]
            if v in {None, ""}:
                cdat[j] = cdat[j-1]
    data = list(zip_longest(*data))
    cr(top, bot).Value = data


In [24]:
fixdata()

In [25]:
with screen_lock(xl):
    for c in ws.Columns:
        v = c.Cells(1,1).Value
        if v is None:
            break
        elif v in {"Time", "Elapsed"}:
            continue
        elif "condenser" not in c.Cells(1,1).Value:
            c.Hidden = True

In [26]:
op_top = cells.Find("condenser.output")
op_bot = op_top.End(xlc.xlDown)
pv_top = cells.Find("condenser.pv")
pv_bot = pv_top.End(xlc.xlDown)

el = cells.Find("Elapsed")
el.Offset(1,2).EntireColumn.Insert()
el2 = el.Offset(2,2)
el2.Value = "=%s/3600"%el.Offset(2,1).GetAddress(0,0)
el_bot = el2.Offset(pv_bot.Row-pv_top.Row, 1)
afr = cr(el2, el_bot)
el2.AutoFill(afr)


True

In [27]:
chart = CreateChart(ws)
chart.ChartStyle = 240
CreateDataSeries(chart, cr(el2, el_bot), cr(pv_top, pv_bot), pv_top)
CreateDataSeries(chart, cr(el2, el_bot), cr(op_top, op_bot), op_top).AxisGroup = xlc.xlSecondary

In [28]:
xax = chart.Axes(xlc.xlCategory, xlc.xlPrimary)
yax = chart.Axes(xlc.xlValue, xlc.xlPrimary)
yax2 = chart.Axes(xlc.xlValue, xlc.xlSecondary)

for ax in (xax, yax, yax2):
    ax.HasTitle = True
    
xax.AxisTitle.Text = "Time"
yax.AxisTitle.Text = "Temp (C)"
yax2.AxisTitle.Text = "Heat Duty (%)"